### Notebook to parse text files to produce cleaned text of RAD decisions

Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0). 

Dataset & Code to be cited as:

Sean Rehaag, "Refugee Appeal Division Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/rad/>.

Notes:

(1) Data Source: Immigration and Refugee Board. In the Fall of 2022, the IRB added the Refugee Law Laboratory to their email distribution list for legal publishers of RAD decisions. The RLL therefore receives new RAD cases as they are released for publication by the IRB. Also, in the fall of 2022 the Immigration and Refugee Board provided the RLL with a full backlog of approximately 116k published decisions from all divisions (RAD, RPD, ID, IAD). 

(2) Unofficial Data: The data are unofficial reproductions. For official versions, please contact the Immigration and Refugee Board. 

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Immigration and Refugee Board.

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, please contact the Immigration and Refugee Board. 

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information.

Acknowledgements: Thanks to Rafael Dolores for coding the parsing scripts.


# Installing Libraries

In [1]:
#!pip install langdetect
#!pip install regex
#!pip install dask

# Importing Libraries

In [2]:
import os
import regex as re 
import pandas as pd
from datetime import datetime
from langdetect import detect, DetectorFactory
from difflib import get_close_matches
import json
from tqdm import tqdm
import chardet
import dask.bag as db
from dask.diagnostics import ProgressBar
import pathlib
from pathlib import Path
from datetime import datetime, timezone

## Declaring Constant
Here, we specify the directories containing our data files.

In [3]:
#DATA_DIRS = ["../RAD Decisions TEXT", "../IRB Decisions - Initial Request - TEXT"]

# For SR:
DATA_DIRS = ["d:/RAD Decisions TEXT/", "d:/IRB Decisions - Initial Request - TEXT/"]

In [4]:
# set seed for langdetect for consistent results and reproducibility
DetectorFactory.seed = 42



## Language Detection
This function determines the language of a given text.

In [5]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return None


## Regular Expression Detector
Functions to parse the date from text files while accounting for several different formats

In [6]:
def original_match_date_patterns(content):
    patterns = {
        "custom": (r"Date (?:of decision|de la décision)\s*\n\s*([A-Za-z]+)\s+(\d{1,2})\.\s*(\d{4})", lambda m: [m.group(1), m.group(2), m.group(3)]),
        "primary": (r"Date (?:of decision|de la décision)\s*(?:Le )?\s*((?:(?:\d{1,2}|1er)\s+[\w]+\s*,?\s*\d{1,4})|\w+\s+\d{1,2}(?:st|nd|rd|th)?\s*,?\s*\d{1,4}|\d{1,2}-\d{1,2}-\d{1,4})", lambda m: m.group(1).replace(',', '').split()),
        "original_decision": (r"Date of decision\s+([\w\s]+),\s+(\d{4})\s+\(original decision\)", lambda m: m.group(1).strip().split() + [m.group(2).strip()]),
        "tribunal": (r"Tribunal\s*\n\s*([\w\s]+?)\s*\n\s*Date of decision", lambda m: m.group(1).replace(',', '').split()),
        "original": (r"Original\s+([\w]+\s+\d{1,2}(?:st|nd|rd|th)?,?\s+\d{4})", lambda m: m.group(1).replace(',', '').split())
    }

    for key, (pattern, process) in patterns.items():
        match = re.search(pattern, content, re.IGNORECASE)
        if match:
            return process(match)
    return None

def match_date_patterns(content):
    # 1. Try your proven patterns first
    result = original_match_date_patterns(content)
    if result:
        return result

    # 2. Fallback: Check lines after the label
    lines = content.splitlines()
    label_regex = re.compile(
        r'^\s*Date (of decision|de la décision)(\s*and reasons|\s*et des motifs)?\s*$', re.IGNORECASE)

    date_regexes = [
        # French style: 1er mai 2015, 24 juillet 2018, Le 24 juillet 2018
        re.compile(r'^(Le\s+)?(\d{1,2}|1er)\s+\w+\s+\d{4}$', re.IGNORECASE),
        # English style: January 21, 2019 or November 5th, 2014
        re.compile(r'^[A-Za-z]+\s+\d{1,2}(st|nd|rd|th)?\,?\s+\d{4}$'),
        # Dashes (rare but sometimes): 10-12-2018
        re.compile(r'^\d{1,2}-\d{1,2}-\d{4}$'),
    ]

    for idx, line in enumerate(lines):
        if label_regex.match(line):
            # Look ahead at the next 3 non-empty lines
            lookahead = 0
            for j in range(idx+1, min(idx+5, len(lines))):
                candidate = lines[j].strip()
                if not candidate:
                    continue
                lookahead += 1
                for reg in date_regexes:
                    m = reg.match(candidate)
                    if m:
                        cleaned = candidate.replace("Le ", "").replace(",", "")
                        return cleaned.split()
                if lookahead >= 3:
                    break
    
    return None

## Date Formatter
Takes detected regular expression and converts into one common format

In [7]:
french_to_english = {
        'janvier': 'January', 'fevrier': 'February', 'mars': 'March', 'avril': 'April',
        'mai': 'May', 'juin': 'June', 'juillet': 'July', 'aout': 'August',
        'septembre': 'September', 'octobre': 'October', 'novembre': 'November', 'decembre': 'December'
}

def correct_month_name(misspelled_month, possibilities=['Janvier','January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], cutoff=0.6):
    correct_months = get_close_matches(misspelled_month, possibilities, n=1, cutoff=cutoff)
    if correct_months:
        corrected_month = correct_months[0]
        # Check if the corrected month is in the French to English mapping
        return french_to_english.get(corrected_month.lower(), corrected_month)
    else:
        return misspelled_month

def correct_year_typo(year):
    if len(year) == 3 and year.startswith("0"):
        return "20" + year[1:]
    return year

def correct_year_typo(year):
    """Corrects year format typos."""
    return "20" + year[1:] if len(year) == 3 and year.startswith("0") else year

def process_numeric_format(parts):
    """Processes numeric date format 'dd-mm-yyyy'."""
    day, month, year = parts[0].split('-')
    year = correct_year_typo(year)
    return datetime(int(year), int(month), int(day)).date().strftime('%Y-%m-%d')

def process_day_first_format(parts, french_month_mapping):
    """Processes dates in 'day month year' format, French or English."""
    day = 1 if parts[0].lower() == '1er' else int(parts[0])

    month = ''
    # Check if month and year are concatenated
    if len(parts) == 2 and not parts[1].isdigit():
        month_year_str = parts[1]
        for i in range(1, len(month_year_str)):
            if month_year_str[i:].isdigit():
                month_str, year_str = month_year_str[:i], month_year_str[i:]
                year = correct_year_typo(year_str)
                month = french_month_mapping.get(month_str.lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o'), month_str.capitalize())
                break
    else:
        month = parts[1].lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o')
        year = correct_year_typo(parts[2])

    if month in french_month_mapping:
        return datetime(int(year), french_month_mapping[month], day).date().strftime('%Y-%m-%d')
    else:
        if isinstance(month, int):
            return datetime(int(year), month, day).date().strftime('%Y-%m-%d')
        
        corrected_month = correct_month_name(month.capitalize())
        try:
            return datetime.strptime(f"{corrected_month} {day} {year}", '%B %d %Y').date().strftime('%Y-%m-%d')
        except ValueError as e:
            print(f"Error parsing date: {e}")
            return None

def process_month_first_format(parts):
    """Processes month first format with possible ordinal suffix."""
    day = 0
    month = ''
    year = ''
    
    if len(parts) == 2 and parts[1].isdigit() and len(parts[1]) > 2:
        
        if parts[1].isdigit() and len(parts[1]) > 4: 
            month = parts[0]
            year_str = parts[1][-4:]
            day_str = parts[1][:-4]
            year = year_str
            day = int(day_str)
            
        elif parts[1].isdigit()and len(parts[1]) > 3: #Year is the second entry
            month_day_str = parts[0]
            for i in range(1, len(month_day_str)):
                if not month_day_str[i].isdigit():
                    day_str, month_str = month_day_str[:i], month_day_str[i:]
                    day = int(day_str)
                    month = french_to_english.get(month_str.lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o'), month_str)
                    parts[0] = month
                    break
            year = parts[1]
        else:
            year_str = parts[1][-4:]
            day_str = parts[1][:-4]
            year = correct_year_typo(year_str)
            day = int(day_str)

    else:
        day = re.sub(r"[^\d]", "", parts[1])
        day = int(day) if day.isdigit() else 1
        year = correct_year_typo(parts[2])
        
    
    try:
        corrected_month = correct_month_name(parts[0].capitalize())
        return datetime.strptime(f"{corrected_month} {day} {year}", '%B %d %Y').date().strftime('%Y-%m-%d')
    except ValueError as e:
        print(f"Error parsing date: {e}")
        return None


## Document Date Extraction
This function searches the given file for the document date using regular expressions, taking into account both French and English texts.

In [8]:
def process_date_parts(parts, french_month_mapping):
    """Determines the correct date processing method based on the format of the parts."""
    if '-' in parts[0]:
        return process_numeric_format(parts)
    elif parts[0].isdigit() or parts[0].lower() == '1er':
        return process_day_first_format(parts, french_month_mapping)
    else:
        return process_month_first_format(parts)

def extract_document_date(content):
    french_month_mapping = {
        'janvier': 1, 'fevrier': 2, 'mars': 3, 'avril': 4,
        'mai': 5, 'juin': 6, 'juillet': 7, 'aout': 8,
        'septembre': 9, 'octobre': 10, 'novembre': 11, 'decembre': 12
    }
    
    parts = match_date_patterns(content)
    
    if not parts:
        return None
    return process_date_parts(parts, french_month_mapping)

## File Processor Helpers

In [9]:
def extract_rad_number(content):
    """Extracts the RAD number from the content, ignoring IAD files."""
    # Check for lines indicating the file should be ignored
    ignore_lines = ["IAD File",
                    "IMMIGRATION APPEAL DIVISION", 
                    "ID File", 
                    "IMMIGRATION DIVISION", 
                    "RPD File", 
                    "RPD file",
                    "REFUGEE PROTECTION DIVISION", 
                    "REFUGEE DIVISION"
                    ]
    
    
    for line in content.splitlines():
     
        if any(ignore_line in line for ignore_line in ignore_lines):       
            return None
                
        sanitized_line = ''.join(c for c in line if c.isprintable()).strip()
    
        if "RAD File" in sanitized_line :
            
            rad_number_match = re.search(r"([A-Z]{2}\d+.\d+)", sanitized_line)
            if rad_number_match:
                return rad_number_match.group(1)
            
            # If RAD is in the next immediate line
            next_line_index = content.splitlines().index(line) + 1
            if next_line_index < len(content.splitlines()):
                next_line = content.splitlines()[next_line_index]
                rad_number_match = re.search(r"([A-Z]{2}\d+-\d+)", next_line)
                if rad_number_match:
                    return rad_number_match.group(1)
              
    return None

def process_file(file_path):

    try:
        """Processes a single file and extracts data."""

        # Use chardet to detect the encoding of the file
        with open(file_path, 'rb') as file:
            raw_data = file.read()
        encoding = chardet.detect(raw_data)['encoding']

        # Read the file with the detected encoding
        with open(file_path, 'r', errors='replace', encoding=encoding) as file:
            content = file.read()

        rad_number = extract_rad_number(content)
        if rad_number:
            lang = detect_language(content)
            document_date = extract_document_date(content)
            scraped_timestamp = datetime.fromtimestamp(os.path.getmtime(file_path), tz=timezone.utc)

            return {
                'citation': rad_number,
                'citation2': '',
                'dataset': 'RAD',
                'name': '',
                'language': lang,
                'document_date': document_date,
                'url': os.path.basename(file_path),
                'scraped_timestamp':  scraped_timestamp,
                'unofficial_text': content,
            }
        return None
    
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        # raise Exception(f"Error processing file {file_path}: {e}") ### uncomment to see error if running in dask
        return None

## Processing Files
This block of code reads each file in the dataset directories to extract the needed information, using the previously defined functions and form a Pandas dataframe.

In [10]:
# Main data processing loop (run in paraellel using Dask)

def process_file_wrapper(file_path):
    if not os.path.basename(file_path).startswith('~'):
        return process_file(file_path)

# Gather all file paths
file_paths = []
for data_dir in DATA_DIRS:
    if os.path.exists(data_dir) and os.path.isdir(data_dir):
        dir_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
        file_paths.extend(dir_files)

# Create a Dask Bag from file paths
file_bag = db.from_sequence(file_paths)

# Use Dask to process files in parallel
with ProgressBar():
    results = file_bag.map(process_file_wrapper).filter(lambda x: x is not None).compute()

# Convert results to a Pandas DataFrame
df = pd.DataFrame(results)

[########################################] | 100% Completed | 249.33 s


In [11]:
df_backup = df.copy()

In [ ]:
df = df_backup.copy()

### Data cleaning
Cleans data to match huggingface dataset

In [ ]:
print ("Starting df", len(df))

# fix dates format
df['document_date'] = pd.to_datetime(df['document_date'], errors='coerce', utc=True)

# remove duplicate rows based on 'unofficial_text' and keep the last one
df = df.sort_values(by='scraped_timestamp', ascending=True)
df = df.drop_duplicates(subset=['unofficial_text'], keep='last').reset_index(drop=True)
print ("After removing duplicate unofficial text", len(df))

# print number of rows with no document_date
print("Number of rows with no document_date:", df['document_date'].isna().sum())

# export to parquet the rows where there is no document_date
df_no_date = df[df['document_date'].isna()]
df_no_date.to_parquet("DATA/df_no_date.parquet", index=False)

# Remove rows where document_date is before 2013
df = df[df['document_date'] >= pd.Timestamp('2013-01-01', tz='UTC')].reset_index(drop=True)

print ("After removing before 2013", len(df)) 

# reset index
df = df.reset_index(drop=True)

df



Starting df 29274
After removing duplicate unofficial text 28847
Number of rows with no document_date: 50
After removing before 2002 and after 2020 28794


,citation,citation2,dataset,name,language,document_date,url,scraped_timestamp,unofficial_text
0,MB7-00112,,RAD,,en,2020-09-15 00:00:00+00:00,MB7-00112a.txt,2023-11-12 22:35:27.041878+00:00,\nRAD File / Dossier de la SAR : MB7-00112\n\n...
1,MB7-00112,,RAD,,fr,2020-09-15 00:00:00+00:00,MB7-00112tf.txt,2023-11-12 22:35:27.195915+00:00,\nDossier de la SAR / RAD File: MB7-00112\n\nH...
2,MB7-03926,,RAD,,fr,2020-10-26 00:00:00+00:00,MB7-03926f.txt,2023-11-12 22:35:27.358429+00:00,\nDossier de la SAR / RAD File : MB7-03926\nMB...
3,MB7-03926,,RAD,,en,2020-10-26 00:00:00+00:00,MB7-03926ta.txt,2023-11-12 22:35:27.492456+00:00,\nRAD File No. / No de dossier de la SAR : MB7...
4,MB7-24221,,RAD,,fr,2021-04-01 00:00:00+00:00,MB7-24221 f.txt,2023-11-12 22:35:27.615982+00:00,\nDossier de la SAR / RAD File : MB7-24221\n\n...
...,...,...,...,...,...,...,...,...,...
28789,VC1-07946,,RAD,,fr,2022-06-17 00:00:00+00:00,VC1-07946 tf.txt,2024-09-18 01:21:42.166912+00:00,\nDossier de la SAR / RAD File: VC1-07946\nVC1...
28790,VC1-07999,,RAD,,en,2022-05-30 00:00:00+00:00,VC1-07999 a.txt,2024-09-18 01:21:42.428914+00:00,\nRAD File / Dossier de la SAR : VC1-07999\n\n...
28791,VC1-07999,,RAD,,fr,2022-05-30 00:00:00+00:00,VC1-07999 tf.txt,2024-09-18 01:21:42.741473+00:00,\nDossier de la SAR / RAD File: VC1-07999\n\nH...
28792,MB8-19805,,RAD,,en,2019-09-09 00:00:00+00:00,MB8-19805a - Dec 2024.txt,2025-01-05 20:08:55.306429+00:00,\nRAD File / Dossier de la SAR : MB8-19805\nMB...


In [14]:
# Clean text of cases
def clean_text(text):

    # remove \xa0
    text = text.replace('\xa0', ' ')

    # Remove multiple whitespaces and preserve paragraphs
    text = '\n'.join([re.sub(r'\s+', ' ', line.strip()) for line in text.split('\n')])
    
    # # Remove single newlines
    # text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Convert multiple newlines to single newlines
    text = re.sub(r'\n+', '\n', text)

    # Remove all strings '\n[Page #]\n' (with # being a number of up to 4 digits 
    text = re.sub(r'\n\[Page \d{1,3}\]\n', ' ', text)
    
    return text

tqdm.pandas()
df['unofficial_text'] = df.unofficial_text.progress_apply(clean_text)

100%|██████████| 28794/28794 [00:39<00:00, 736.30it/s]


In [15]:
# get df_en where language is English
df_en = df[df['language'] == 'en'].reset_index(drop=True)
# get df_fr where language is French
df_fr = df[df['language'] == 'fr'].reset_index(drop=True)

# print how many df_en.citation is duplicated
print("Number of duplicated citations in df_en:", df_en.citation.duplicated().sum())
print("Number of duplicated citations in df_fr:", df_fr.citation.duplicated().sum())

# print how many rows in df_en are duplicated for BOTH citation and document_date
print("Number of duplicated rows in df_en for BOTH citation and document_date:", df_en.duplicated(subset=['citation', 'document_date']).sum())
print("Number of duplicated rows in df_fr for BOTH citation and document_date:", df_fr.duplicated(subset=['citation', 'document_date']).sum())

# Blunt removal of duplicates in df_en and df_fr
df_en = df_en.drop_duplicates(subset=['citation']).reset_index(drop=True)
df_fr = df_fr.drop_duplicates(subset=['citation']).reset_index(drop=True)
# Print the number of rows after removing duplicates
print("Number of rows in df_en after removing all duplicate citations:", len(df_en))
print("Number of rows in df_fr after removing duplicate citations:", len(df_fr))

# number of rows in df_en that does not have an identical citation in df_fr
print(
    "Num rows in df_en with no citation match in df_fr",
    df_en[~df_en['citation'].isin(df_fr['citation'])].reset_index(drop=True).shape[0]
)
# number of rows in df_fr that does not have an identical citation in df_en
print(
    "Num rows in df_fr with no citation match in df_en",
    df_fr[~df_fr['citation'].isin(df_en['citation'])].reset_index(drop=True).shape[0]
)



# for df_en add "_en" to end of all cols
df_en.columns = [col + "_en" for col in df_en.columns]
# for df_fr add "_fr" to end of all cols
df_fr.columns = [col + "_fr" for col in df_fr.columns]

# merge df_en and df_fr on citation_en and citation_fr, keeping only matches
df = pd.merge(df_en, df_fr, left_on='citation_en', right_on='citation_fr', how='inner')

# rename dataset_en to dataset, and drop unused cols
df = df.rename(columns={'dataset_en': 'dataset'})
df = df.drop(columns=['dataset_fr', 'language_en', 'language_fr'])


# print len of final df
print("Number of rows in final df:", len(df))
print("*********** AT SOME POINT RECONSIDER THIS ***********")
df.head(5)

Number of duplicated citations in df_en: 597
Number of duplicated citations in df_fr: 575
Number of duplicated rows in df_en for BOTH citation and document_date: 25
Number of duplicated rows in df_fr for BOTH citation and document_date: 21
Number of rows in df_en after removing all duplicate citations: 13841
Number of rows in df_fr after removing duplicate citations: 13781
Num rows in df_en with no citation match in df_fr 143
Num rows in df_fr with no citation match in df_en 83
Number of rows in final df: 13698
*********** AT SOME POINT RECONSIDER THIS ***********


,citation_en,citation2_en,dataset,name_en,document_date_en,url_en,scraped_timestamp_en,unofficial_text_en,citation_fr,citation2_fr,name_fr,document_date_fr,url_fr,scraped_timestamp_fr,unofficial_text_fr
0,MB7-00112,,RAD,,2020-09-15 00:00:00+00:00,MB7-00112a.txt,2023-11-12 22:35:27.041878+00:00,\nRAD File / Dossier de la SAR : MB7-00112\nPr...,MB7-00112,,,2020-09-15 00:00:00+00:00,MB7-00112tf.txt,2023-11-12 22:35:27.195915+00:00,\nDossier de la SAR / RAD File: MB7-00112\nHui...
1,MB7-03926,,RAD,,2020-10-26 00:00:00+00:00,MB7-03926ta.txt,2023-11-12 22:35:27.492456+00:00,\nRAD File No. / No de dossier de la SAR : MB7...,MB7-03926,,,2020-10-26 00:00:00+00:00,MB7-03926f.txt,2023-11-12 22:35:27.358429+00:00,\nDossier de la SAR / RAD File : MB7-03926\nMB...
2,MB7-24221,,RAD,,2021-04-01 00:00:00+00:00,MB7-24221ta.txt,2023-11-12 22:35:27.758493+00:00,\nRAD File No. / No de dossier de la SAR : MB7...,MB7-24221,,,2021-04-01 00:00:00+00:00,MB7-24221 f.txt,2023-11-12 22:35:27.615982+00:00,\nDossier de la SAR / RAD File : MB7-24221\nHu...
3,MB8-01089,,RAD,,2020-10-01 00:00:00+00:00,MB8-01089a.txt,2023-11-12 22:35:27.935519+00:00,\nRAD File / Dossier de la SAR : MB8-01089\nPr...,MB8-01089,,,2020-10-01 00:00:00+00:00,MB8-01089tf.txt,2023-11-12 22:35:28.081037+00:00,\nDossier de la SAR / RAD File: MB8-01089\nHui...
4,MB8-02514,,RAD,,2020-07-28 00:00:00+00:00,MB8-02514ta.txt,2023-11-12 22:35:28.336570+00:00,\nRAD File No. / No de dossier de la SAR : MB8...,MB8-02514,,,2020-07-28 00:00:00+00:00,MB8-02514f.txt,2023-11-12 22:35:28.209052+00:00,\nDossier de la SAR / RAD File : MB8-02514\nHu...


In [16]:
# how many rows in df have document_date_en and document_date_fr that are not the same
print("Number of rows in df with different document_date_en and document_date_fr:", 
      df[df['document_date_en'] != df['document_date_fr']].shape[0])

Number of rows in df with different document_date_en and document_date_fr: 30


In [17]:
df[df['document_date_en'] != df['document_date_fr']]

,citation_en,citation2_en,dataset,name_en,document_date_en,url_en,scraped_timestamp_en,unofficial_text_en,citation_fr,citation2_fr,name_fr,document_date_fr,url_fr,scraped_timestamp_fr,unofficial_text_fr
685,TB8-11442,,RAD,,2020-10-15 00:00:00+00:00,TB8-11442a.txt,2023-11-12 22:39:25.142694+00:00,\nRAD File / Dossier de la SAR : TB8-11442\nPr...,TB8-11442,,,2019-10-02 00:00:00+00:00,3074172.txt,2023-11-13 16:31:06.400718+00:00,\nDossier de la SAR / RAD File: TB8-11442\nHui...
2095,VB4-00040,,RAD,,2014-01-27 00:00:00+00:00,1558524.txt,2023-11-13 02:01:46.774406+00:00,\nRAD File No. / N° de dossier de la SAR : VB4...,VB4-00040,,,2014-04-25 00:00:00+00:00,1797760.txt,2023-11-13 02:20:42.429666+00:00,Commission de l'immigration\net du statut de r...
2238,MB4-01560,,RAD,,2014-06-23 00:00:00+00:00,1577631.txt,2023-11-13 02:03:13.807974+00:00,\nRAD File No. / No de dossier de la SAR : MB4...,MB4-01560,,,2014-10-30 00:00:00+00:00,1708532.txt,2023-11-13 02:05:23.260118+00:00,Commission de l'immigration\net du statut de r...
2248,MB4-00454,,RAD,,2014-04-22 00:00:00+00:00,1579925.txt,2023-11-13 02:03:44.976657+00:00,\nRAD File No. / N° de dossier de la SAR : MB4...,MB4-00454,,,2016-02-11 00:00:00+00:00,2229678.txt,2023-11-13 03:24:39.555093+00:00,\nN° de dossier de la SAR/RAD File No MB4-0045...
2372,MB4-02818,,RAD,,2014-08-15 00:00:00+00:00,1700160.txt,2023-11-13 02:04:49.237288+00:00,\nRAD File No. / N° de dossier de la SAR : MB4...,MB4-02818,,,2014-10-02 00:00:00+00:00,1700169.txt,2023-11-13 02:04:49.779948+00:00,Commission de l'immigration\net du statut de r...
2857,MB4-03651,,RAD,,2015-04-09 00:00:00+00:00,1788560.txt,2023-11-13 02:16:43.252584+00:00,Immigration and\nRefugee Board of Canada\nRefu...,MB4-03651,,,2016-01-15 00:00:00+00:00,2324261.txt,2023-11-13 15:00:17.301358+00:00,Commission de l'immigration\net du statut de r...
3314,MB5-00725,,RAD,,2015-04-24 00:00:00+00:00,1820460.txt,2023-11-13 02:23:56.325596+00:00,\nImmigration and\nRefugee Board of Canada\nRe...,MB5-00725,,,2014-04-24 00:00:00+00:00,1830350.txt,2023-11-13 02:30:22.303826+00:00,\nCommission de l'immigration\net du statut de...
3529,TB4-07173,,RAD,,2015-01-28 00:00:00+00:00,1825358.txt,2023-11-13 02:27:52.902273+00:00,Immigration and\nRefugee Board of Canada\nRefu...,TB4-07173,,,2016-05-18 00:00:00+00:00,2305219.txt,2023-11-13 14:34:14.695941+00:00,Commission de l'immigration\net du statut de r...
3853,TB3-07972,,RAD,,2014-01-05 00:00:00+00:00,1833703.txt,2023-11-13 02:31:44.877056+00:00,\nRAD File No. / N° de dossier de la SAR :\nTB...,TB3-07972,,,2015-09-02 00:00:00+00:00,2261421.txt,2023-11-13 09:20:26.227039+00:00,Commission de l'immigration\net du statut de r...
4576,TB4-03278,,RAD,,2015-11-04 00:00:00+00:00,2108488.txt,2023-11-13 03:21:28.621365+00:00,Immigration and\nRefugee Board of Canada\nRefu...,TB4-03278,,,2014-11-05 00:00:00+00:00,1831966.txt,2023-11-13 02:31:30.483250+00:00,Commission de l'immigration\net du statut de r...


In [18]:
# are there any duplicate citations_en in df?
print("Number of duplicated citations_en in df:", df.citation_en.duplicated().sum())
# are there any duplicate citations_fr in df?
print("Number of duplicated citations_fr in df:", df.citation_fr.duplicated().sum())

# are there any blank or null or nan citations_en in df?
print("Number of blank or null or nan citations_en in df:", df['citation_en'].isnull().sum())
# are there any blank or null or nan citations_fr in df?
print("Number of blank or null or nan citations_fr in df:", df['citation_fr'].isnull().sum())

Number of duplicated citations_en in df: 0
Number of duplicated citations_fr in df: 0
Number of blank or null or nan citations_en in df: 0
Number of blank or null or nan citations_fr in df: 0


In [ ]:
# # To compare duplicates
# import difflib

# # Get the two texts
# texts = df_fr[df_fr.duplicated(subset=['citation', 'document_date'], keep=False)].unofficial_text.iloc[[0,1]]

# # Compute the diff
# diff = difflib.unified_diff(
#     texts.iloc[0].splitlines(),
#     texts.iloc[1].splitlines(),
#     fromfile='text0',
#     tofile='text1',
#     lineterm=''
# )

# # Print the diff
# for line in diff:
#     print(line)

### Exports

In [ ]:
# export cleaned df to jsonl
out_path_parsed = pathlib.Path('DATA/rad_cases.jsonl')
df.to_json(out_path_parsed, orient='records', lines=True)

# export cleaned df to parquet
out_path_parsed = pathlib.Path('DATA/rpd_cases.parquet')

In [ ]:
# export to Mongo # NEED TAILSCALE ON
from pymongo import MongoClient
from datetime import timezone
import numpy as np

MONGO_URL = 'mongodb://rll-tr7970x:27017/'
MONGO_DB = 'case-scraping'
MONGO_COLLECTION = 'rad-scrapes'
client = MongoClient(MONGO_URL, tz_aware=True, tzinfo=timezone.utc)
db = client[MONGO_DB]
collection = db[MONGO_COLLECTION]

# drop existing collection if it exists
if collection.count_documents({}) > 0:
    collection.drop()

def strip_nulls(recs):
    return [{k: v for k, v in rec.items() if v is not None} for rec in recs]

def sanitize_records(recs):
    """Return a NEW list of dicts that PyMongo can store."""
    return [
        {k: _clean_scalar(v) for k, v in rec.items()}
        for rec in recs
    ]

def _clean_scalar(v):
    # 1) All “missing” markers: <NA>, NaN, NaT  →  None
    if pd.isna(v):
        return None
    
    # 2) pandas Timestamp  →  native datetime with UTC tzinfo
    if isinstance(v, pd.Timestamp):
        return v.to_pydatetime().astimezone(timezone.utc)
    
    # 3) NumPy scalar (np.int64, np.float64, …)  →  Python int/float
    if isinstance(v, np.generic):
        return v.item()
    
    return v

records = df.to_dict(orient="records")
clean_docs = strip_nulls(sanitize_records(records))
result = collection.insert_many(clean_docs)


100%|██████████| 12/12 [00:01<00:00,  7.93it/s]


In [15]:
# replace 'other' column with empty string for huggingface
df['other'] = ''

# eport to parquet for hugginface
out_path_parquet = pathlib.Path('DATA/rad_cases.parquet')
df.to_parquet(out_path_parquet)

In [16]:
# Direct to HF
out_path_HF = pathlib.Path('d:/AI-Projects/canadian-legal-data/RAD/train.parquet')
df.to_parquet(out_path_HF)

# NOTES:

Note that the try/except print doesn't necessarily work if running in jupyter due to limitations 
in DASK. So if you need to know whether there are errors, uncomment the raise error thing.


TC1-10832tf.txt throws an error for some reason not sure why.